In [ ]:
%pip install -r requirements.txt

In [35]:
import dagshub
import datetime
import mlflow
import torch
import time
import os
import shutil
import random
import yaml
import pandas as pd
from pathlib import Path
from roboflow import Roboflow
from ultralytics import YOLO, settings

In [ ]:
rf = Roboflow(api_key="aRTRw0ORiLGJdj7RF2gR")
project = rf.workspace("abiya-thesis").project("tennis-player-and-tennis-ball-ei0vm")
version = project.version(6)
dataset = version.download("yolov12", location="datasets/object-detection")

In [2]:
rf = Roboflow(api_key="aRTRw0ORiLGJdj7RF2gR")
project = rf.workspace("abiya-thesis").project("tennis-player-and-tennis-ball-ei0vm")
version = project.version(7)
dataset = version.download("yolov12",location="datasets/object-detection-tile")

loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to datasets/object-detection-tile in yolov12:: 100%|██████████| 21068/21068 [00:07<00:00, 2822.45it/s]


In [3]:
dagshub.init(repo_owner='abiyamf', repo_name='courtPlay', mlflow=True)
settings.update({"mlflow": True})

Accessing as nugrohorahmanto24

Initialized MLflow to track repo "abiyamf/courtPlay"

Repository abiyamf/courtPlay initialized!

In [74]:
# ==========================
# 🔧 Global Config
# ==========================
os.environ["MLFLOW_KEEP_RUN_ACTIVE"] = "True"

time_now = datetime.datetime.now().strftime("%m-%d_%H-%M")
gpu_name = torch.cuda.get_device_name(0)

# Project Info
projectType = "object-detection"
datasetVersion = "6"
yoloVersion = "yolo11"
modelSize = "n"

# Training Params
epochs = 50
batch = 16
imgsz = 720
data_path = "datasets/object-detection/data.yaml"
project_path = f"results/object-detection/{modelSize}"

# Toggle custom augmentation
use_custom_aug = False

In [75]:
def format_duration(seconds: float) -> str:
    """Convert seconds into 'Hh Mm Ss' format."""
    hours = int(seconds // 3600)
    minutes = int((seconds % 3600) // 60)
    secs = int(seconds % 60)
    return f"{hours}h {minutes}m {secs}s"

def save_class_metrics(val_results):
    # Ambil nama class
    names = val_results.names

    # Data per-class
    data = {
        "class": [names[i] for i in range(len(names))],
        "precision": val_results.box.p.tolist(),
        "recall": val_results.box.r.tolist(),
        "mAP50": val_results.box.ap50.tolist(),
        "mAP50-95": val_results.box.maps.tolist(),
    }

    # Tambahkan baris agregat "all"
    all_row = {
        "class": "all",
        "precision": float(val_results.box.mp),
        "recall": float(val_results.box.mr),
        "mAP50": float(val_results.box.map50),
        "mAP50-95": float(val_results.box.map),
    }

    for key in data.keys():
        data[key].insert(0, all_row[key])

    # Buat DataFrame dan simpan
    df = pd.DataFrame(data)
    csv_path = os.path.join(str(val_results.save_dir), "class_metrics.csv")
    df.to_csv(csv_path, index=False)

    return csv_path

In [73]:
# ==========================
# 🚀 Training Routine
# ==========================
model = YOLO(f"{yoloVersion}{modelSize}.pt")
mlflow.set_experiment(f"courtPlay-{modelSize}")

start_time = time.time()

# Set augmentations (default vs custom)
augmentations = {}
if use_custom_aug:
    augmentations = {
        "mosaic": 0.4,
        "scale": 0.25,
        "translate": 0.05,
        "hsv_s": 0.4,
        "hsv_v": 0.2,
        "mixup": 0.1,
        "flipud": 0.0,
        "cutmix": 0.0,
        "shear": 0.0,
        "perspective": 0.0,
    }

with mlflow.start_run(run_name=f"{time_now}"):

    # Log global parameters
    mlflow.log_params({
        "project type": projectType,
        "dataset version": datasetVersion,
        "yolo version": yoloVersion,
        "model size": modelSize,
        "gpu": gpu_name,
        "epochs": epochs,
        "batch": batch,
        "imgsz": imgsz,
        "use_custom_aug": use_custom_aug,
    })

    # Train
    results = model.train(
        data=data_path,
        epochs=epochs,
        imgsz=imgsz,
        batch=batch,
        name=f"{time_now}",
        project=project_path,
        exist_ok=True,
        **augmentations
    )

    # Log training time
    mlflow.log_params({
        "training time": format_duration(time.time() - start_time),
    })

    # Run validation + log class metrics
    mlflow.log_artifact(save_class_metrics(results), artifact_path="class_metrics")

mlflow.end_run()

New https://pypi.org/project/ultralytics/8.3.204 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.179  Python-3.10.18 torch-2.8.0+cu129 CUDA:0 (NVIDIA GeForce RTX 3060, 12288MiB)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=datasets/object-detection/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=2, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=10-05_14-32, nbs=64, nms=False, o

train: Scanning C:\Users\User\Documents\ABIYA NITIP\TelU-Tubes-BisnisDigital-CourtPlay-AIDevelopment\ObjectDetection\datasets\object-detection\train\labels.cache... 3686 images, 2 backgrounds, 0 corrupt: 100%|██████████| 3686/3686 [00:00<?, ?it/s]


val: Fast image access  (ping: 0.00.0 ms, read: 1892.0464.9 MB/s, size: 197.3 KB)


val: Scanning C:\Users\User\Documents\ABIYA NITIP\TelU-Tubes-BisnisDigital-CourtPlay-AIDevelopment\ObjectDetection\datasets\object-detection\valid\labels.cache... 527 images, 1 backgrounds, 0 corrupt: 100%|██████████| 527/527 [00:00<?, ?it/s]


Plotting labels to results\object-detection\n\10-05_14-32\labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.001667, momentum=0.9) with parameter groups 81 weight(decay=0.0), 88 weight(decay=0.0005), 87 bias(decay=0.0)
MLflow: logging run_id(73687966e7e54a8d9d9adf3836c47469) to https://dagshub.com/abiyamf/courtPlay.mlflow
MLflow: disable with 'yolo settings mlflow=False'
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to results\object-detection\n\10-05_14-32
Starting training for 2 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

        2/2      2.71G      1.086      0.741     0.9196         46        640: 100%|██████████| 231/231 [00:33<00:00,  6.82it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 17/17 [00:02<00:00,  7.62it/s]

                   all        527       1962      0.811      0.691      0.696      0.416



2 epochs completed in 0.023 hours.
Optimizer stripped from results\object-detection\n\10-05_14-32\weights\last.pt, 5.4MB
Optimizer stripped from results\object-detection\n\10-05_14-32\weights\best.pt, 5.4MB

Validating results\object-detection\n\10-05_14-32\weights\best.pt...
Ultralytics 8.3.179  Python-3.10.18 torch-2.8.0+cu129 CUDA:0 (NVIDIA GeForce RTX 3060, 12288MiB)
YOLO11n summary (fused): 100 layers, 2,582,542 parameters, 0 gradients, 6.3 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 17/17 [00:02<00:00,  5.93it/s]


                   all        527       1962      0.807      0.691      0.696      0.415
                  ball        408        414      0.668      0.394      0.402      0.117
                player        526       1548      0.947      0.989      0.989      0.714
Speed: 0.1ms preprocess, 1.1ms inference, 0.0ms loss, 1.4ms postprocess per image
Results saved to results\object-detection\n\10-05_14-32
MLflow: mlflow run still alive, remember to close it using mlflow.end_run()
MLflow: results logged to https://dagshub.com/abiyamf/courtPlay.mlflow
MLflow: disable with 'yolo settings mlflow=False'
🏃 View run 10-05_14-32 at: https://dagshub.com/abiyamf/courtPlay.mlflow/#/experiments/6/runs/73687966e7e54a8d9d9adf3836c47469
🧪 View experiment at: https://dagshub.com/abiyamf/courtPlay.mlflow/#/experiments/6
